In [9]:
import os
import pandas as pd
import numpy as np
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()
import pathlib as Path
#!pip install pandas-datareader
import pandas_datareader as pdr
from datetime import datetime

%matplotlib inline

In [10]:
# FOR HENNA ONLY
import os
os.chdir('/Users/hennasingh/Jupyter-Workspace/PROJECT1/Project1/notebooks')

In [15]:
# set time frame
timeframe = '1D'

# start and 'end' date for covid stock market 


start_date_covid = datetime(2017,11,1)
end_date_covid = datetime(2019,11,1)

# start and end date for 2008 recession 
start_date_2008 = datetime(2005,7,1)
end_date_2008 = datetime(2007,7,1)

# set entertainment tickers (per industry)
ent_tickers = ['IMAX','DIS','NFLX','CMCSA','SNE']
fast_food_tickers = ['MCD','WEN','YUM','DPZ','JACK']
tech_tickers = ['AMZN', 'AAPL', 'MSFT', 'GOOG', 'INTC']
tickers_travel_airline = ['EXPE','BKNG','AAL','AAP','LUV']
energy_tickers = ['XOM','BP','COP','CVX', 'TOT']

# get stock data from covid
entertainment_tickers_covid = pdr.DataReader(ent_tickers,'yahoo',start_date_covid,end_date_covid)
fastfood_tickers_covid = pdr.DataReader(fast_food_tickers,'yahoo',start_date_covid,end_date_covid)
covid_tech_ticker = pdr.DataReader(tech_tickers,'yahoo',start_date_covid,end_date_covid)
tra_air_covid_ticker = pdr.DataReader(tickers_travel_airline,'yahoo',start_date_covid,end_date_covid)
energy_covid_ticker = pdr.DataReader(energy_tickers,'yahoo',start_date_covid,end_date_covid)
sp500_covid_ticker = pdr.DataReader('^GSPC','yahoo',start_date_covid,end_date_covid)


# get stock data from 2008 recession
entertainment_tickers_2008 = pdr.DataReader(ent_tickers,'yahoo',start_date_2008,end_date_2008)
fastfood_tickers_2008 = pdr.DataReader(fast_food_tickers,'yahoo',start_date_2008,end_date_2008)
tech_ticker2008 = pdr.DataReader(tech_tickers,'yahoo',start_date_2008,end_date_2008)
tra_air_ticker2008 = pdr.DataReader(tickers_travel_airline,'yahoo',start_date_2008,end_date_2008)
energy_ticker2008 = pdr.DataReader(energy_tickers,'yahoo',start_date_2008,end_date_2008)
sp500_2008_ticker = pdr.DataReader('^GSPC','yahoo',start_date_2008,end_date_2008)

## Weight Optimization for Entertainment Industry

### Dataframe with COVID Closing Prices - Entertainment

In [18]:
# create new covid and 2008 dataframe to store ckosing prices of each stock
covid_closingprices_e = pd.DataFrame()

# get closing prices for all tickers
for ticker in ent_tickers:
    covid_closingprices_e[ticker]=entertainment_tickers_covid['Adj Close'][ticker]
    
# drop time component on index
covid_closingprices_e.index = covid_closingprices_e.index.date
    
covid_closingprices_e.head()

,IMAX,DIS,NFLX,CMCSA,SNE
2017-11-01,24.150000,95.538605,198.000000,33.865295,42.838394
2017-11-02,24.100000,94.882591,199.320007,32.992382,44.166637
2017-11-03,24.100000,95.162369,200.009995,33.489845,44.087933
2017-11-06,24.350000,97.091850,200.130005,33.358440,45.130848
2017-11-07,24.299999,98.027657,195.889999,34.156265,45.268593


### Dataframe with 2008 Closing Prices - Entertainment

In [20]:
closingprices2008_e = pd.DataFrame()

# get closing prices for all tickers
for ticker in ent_tickers:
    closingprices2008_e[ticker]=entertainment_tickers_2008['Adj Close'][ticker]
    
# drop time component on index
closingprices2008_e.index = closingprices2008_e.index.date
    
closingprices2008_e.head()

,IMAX,DIS,NFLX,CMCSA,SNE
2005-07-01,9.93,20.371153,2.364286,8.012370,30.189739
2005-07-05,9.72,20.330553,2.348572,8.015000,29.979908
2005-07-06,9.90,20.127573,2.390000,7.915007,30.285908
2005-07-07,9.84,19.851521,2.418571,7.917641,30.233459
2005-07-08,10.03,20.298075,2.387143,8.070257,30.495743


### Entertainment Optimized Weights - COVID

In [22]:
# daily returns
ent_covid_dr = covid_closingprices_e.pct_change().dropna()
ent_returns2008 = closingprices2008_e.pct_change().dropna()

In [23]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ent_covid = np.zeros((number_ports, len(covid_closingprices_e.columns)))
returns_array_ent_covid = np.zeros(number_ports)
volatility_array_ent_covid = np.zeros(number_ports)
sharpe_array_ent_covid = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ent_covid[index,:] = weights

    #expected returns
    returns_array_ent_covid[index] = np.sum((ent_covid_dr.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ent_covid[index] = np.sqrt(np.dot(weights.T, np.dot(ent_covid_dr.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ent_covid[index] = returns_array_ent_covid[index]/volatility_array_ent_covid[index]

In [24]:
sharpe_array_ent_covid.max()

1.0257017499384444

In [25]:
index_max_shape_covid_ent = sharpe_array_ent_covid.argmax()
index_max_shape_covid_ent

54101

In [28]:
weights_array_covid_ent = all_weights_ent_covid[index_max_shape_covid_ent,:]
weights_array_covid_ent

array([0.00112396, 0.5000311 , 0.10987238, 0.12220884, 0.26676372])

In [34]:
weights_array_covid_ent = np.array([0.00112396, 0.5000311 , 0.10987238, 0.12220884, 0.26676372])

### Entertainment Optimized Weights - 2008

In [29]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ent_2008 = np.zeros((number_ports, len(closingprices2008_e.columns)))
returns_array_ent_2008 = np.zeros(number_ports)
volatility_array_ent_2008 = np.zeros(number_ports)
sharpe_array_ent_2008 = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ent_2008[index,:] = weights

    #expected returns
    returns_array_ent_2008[index] = np.sum((ent_returns2008.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ent_2008[index] = np.sqrt(np.dot(weights.T, np.dot(ent_returns2008.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ent_2008[index] = returns_array_ent_2008[index]/volatility_array_ent_2008[index]

In [30]:
sharpe_array_ent_2008.max()

1.3215721366100446

In [31]:
index_max_shape_2008_ent = sharpe_array_ent_2008.argmax()
index_max_shape_2008_ent

31710

In [32]:
weights_array_2008_ent = all_weights_ent_2008[index_max_shape_2008_ent,:]
weights_array_2008_ent

array([0.00062007, 0.38479239, 0.06010945, 0.32760128, 0.22687681])

In [35]:
weights_array_2008_ent = np.array([0.00062007, 0.38479239, 0.06010945, 0.32760128, 0.22687681])

## Weight Optimization for Fast Food Industry


### Dataframe with COVID Closing Prices - Fast Food

In [39]:
# create new covid and 2008 dataframe to store ckosing prices of each stock
covid_closingprices_ff = pd.DataFrame()

# get closing prices for all tickers
for ticker in fast_food_tickers:
    covid_closingprices_ff[ticker]=fastfood_tickers_covid['Adj Close'][ticker]
    
# drop time component on index
covid_closingprices_ff.index = covid_closingprices_ff.index.date
    
covid_closingprices_ff.head()

,MCD,WEN,YUM,DPZ,JACK
2017-11-01,153.664261,14.158257,70.209763,172.127960,96.140327
2017-11-02,155.262131,14.356076,74.782715,172.943649,95.620705
2017-11-03,155.770111,14.148840,74.555954,172.798019,97.018974
2017-11-06,157.081711,14.064059,75.462990,168.447495,96.688301
2017-11-07,157.728241,13.885078,76.540092,165.184570,95.601814


### Dataframe with 2008 Closing Prices - Fast Food

In [40]:
closingprices2008_ff = pd.DataFrame()

# get closing prices for all tickers
for ticker in fast_food_tickers:
    closingprices2008_ff[ticker]=fastfood_tickers_2008['Adj Close'][ticker]
    
# drop time component on index
closingprices2008_ff.index = closingprices2008_ff.index.date
    
closingprices2008_ff.head()

,MCD,WEN,YUM,DPZ,JACK
2005-07-01,17.553865,11.605938,13.830293,10.626073,16.800085
2005-07-05,17.572720,11.562579,13.776560,10.823838,16.939381
2005-07-06,17.403086,11.562579,13.736261,10.611604,16.961842
2005-07-07,17.591564,11.468626,13.814171,10.746663,16.858500
2005-07-08,17.999943,11.331330,13.935070,11.007123,17.119106


### Fast Food Optimized Weights - COVID

In [41]:
# daily returns
ff_covid_dr = covid_closingprices_ff.pct_change().dropna()
ff_returns2008 = closingprices2008_ff.pct_change().dropna()

In [43]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ff_covid = np.zeros((number_ports, len(covid_closingprices_ff.columns)))
returns_array_ff_covid = np.zeros(number_ports)
volatility_array_ff_covid = np.zeros(number_ports)
sharpe_array_ff_covid = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ff_covid[index,:] = weights

    #expected returns
    returns_array_ff_covid[index] = np.sum((ff_covid_dr.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ff_covid[index] = np.sqrt(np.dot(weights.T, np.dot(ff_covid_dr.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ff_covid[index] = returns_array_ff_covid[index]/volatility_array_ff_covid[index]

In [47]:
sharpe_array_ff_covid.max()

1.2567695791189286

In [48]:
index_max_shape_covid_ff = sharpe_array_ff_covid.argmax()
weights_array_covid_ff = all_weights_ff_covid[index_max_shape_covid_ff,:]
weights_array_covid_ff

array([5.57594323e-02, 2.25627731e-01, 4.39993918e-01, 2.78455724e-01,
       1.63195337e-04])

In [49]:
weights_array_covid_ff = np.array([5.57594323e-02, 2.25627731e-01, 4.39993918e-01, 2.78455724e-01,
       1.63195337e-04])

### Fast Food Optimized Weights - 2008

In [50]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ff_2008 = np.zeros((number_ports, len(closingprices2008_ff.columns)))
returns_array_ff_2008 = np.zeros(number_ports)
volatility_array_ff_2008 = np.zeros(number_ports)
sharpe_array_ff_2008 = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ff_2008[index,:] = weights

    #expected returns
    returns_array_ff_2008[index] = np.sum((ff_returns2008.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ff_2008[index] = np.sqrt(np.dot(weights.T, np.dot(ff_returns2008.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ff_2008[index] = returns_array_ff_2008[index]/volatility_array_ff_2008[index]

In [54]:
sharpe_array_ff_2008.max()

1.7902943283052932

In [55]:
index_max_shape_2008_ff = sharpe_array_ff_2008.argmax()
weights_array_2008_ff = all_weights_ff_2008[index_max_shape_2008_ff,:]
weights_array_2008_ff

array([0.63961816, 0.01055246, 0.01573766, 0.08464276, 0.24944896])

In [58]:
weights_array_2008_ff = np.array([0.63961816, 0.01055246, 0.01573766, 0.08464276, 0.24944896])

## Weight Optimization for tech Industry

### Dataframe with COVID Closing Prices -TECH

In [59]:
# create new covid and 2008 dataframe to store ckosing prices of each stock
covid_closingprices_tech = pd.DataFrame()

# get closing prices for all tickers
for ticker in tech_tickers:
    covid_closingprices_tech[ticker]=covid_tech_ticker['Adj Close'][ticker]
    
# drop time component on index
covid_closingprices_tech.index = covid_closingprices_tech.index.date
    
covid_closingprices_tech.head()

,AMZN,AAPL,MSFT,GOOG,INTC
2017-11-01,1103.680054,39.681473,79.330177,1025.500000,43.120304
2017-11-02,1094.219971,39.971542,80.159912,1025.579956,43.480331
2017-11-03,1111.599976,41.015358,80.245743,1032.479980,42.778736
2017-11-06,1120.660034,41.431458,80.560478,1025.900024,43.366554
2017-11-07,1123.170044,41.564606,80.369720,1033.329956,43.440838


### Dataframe with 2008 Closing Prices - TECH

In [62]:
closingprices2008_tech = pd.DataFrame()

# get closing prices for all tickers
for ticker in tech_tickers:
    closingprices2008_tech[ticker]=tech_ticker2008['Adj Close'][ticker]
    
# drop time component on index
closingprices2008_tech.index = closingprices2008_tech.index.date
    
closingprices2008_tech.head()

,AMZN,AAPL,MSFT,GOOG,INTC
2005-07-01,32.910000,1.115395,17.850933,145.081207,16.874496
2005-07-05,34.080002,1.160622,18.045984,147.302872,17.177095
2005-07-06,34.060001,1.142592,17.843708,145.215698,17.061209
2005-07-07,33.959999,1.149926,17.807590,147.218185,17.099836
2005-07-08,34.740002,1.168873,18.125446,147.561905,17.563387


### TECH Optimized Weights - COVID

In [63]:
# daily returns
tech_covid_dr = covid_closingprices_tech.pct_change().dropna()
tech_returns2008 = closingprices2008_tech.pct_change().dropna()

In [64]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_tech_covid = np.zeros((number_ports, len(covid_closingprices_tech.columns)))
returns_array_tech_covid = np.zeros(number_ports)
volatility_array_tech_covid = np.zeros(number_ports)
sharpe_array_tech_covid = np.zeros(number_ports)



for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_tech_covid[index,:] = weights

    #expected returns
    returns_array_tech_covid[index] = np.sum((tech_covid_dr.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_tech_covid[index] = np.sqrt(np.dot(weights.T, np.dot(tech_covid_dr.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_tech_covid[index] = returns_array_tech_covid[index]/volatility_array_tech_covid[index]

In [68]:
sharpe_array_tech_covid.max()

1.2898301658855291

In [69]:
index_max_shape_covid_tech = sharpe_array_tech_covid.argmax()
weights_array_covid_tech = all_weights_tech_covid[index_max_shape_covid_tech,:]
weights_array_covid_tech

array([0.05831981, 0.07284522, 0.83499035, 0.01689324, 0.01695139])

In [71]:
weights_array_covid_tech = np.array([0.05831981, 0.07284522, 0.83499035, 0.01689324, 0.01695139])

### TECH Optimized Weights - 2008

In [70]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_tech_2008 = np.zeros((number_ports, len(closingprices2008_tech.columns)))
returns_array_tech_2008 = np.zeros(number_ports)
volatility_array_tech_2008 = np.zeros(number_ports)
sharpe_array_tech_2008 = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_tech_2008[index,:] = weights

    #expected returns
    returns_array_tech_2008[index] = np.sum((tech_returns2008.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_tech_2008[index] = np.sqrt(np.dot(weights.T, np.dot(tech_returns2008.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_tech_2008[index] = returns_array_tech_2008[index]/volatility_array_tech_2008[index]

In [72]:
sharpe_array_tech_2008.max()

2.0416605531056193

In [73]:
index_max_shape_2008_tech = sharpe_array_tech_2008.argmax()
weights_array_2008_tech = all_weights_tech_2008[index_max_shape_2008_tech,:]
weights_array_2008_tech

array([2.18892709e-01, 5.75050292e-01, 3.97318093e-02, 1.66125078e-01,
       2.00111131e-04])

In [74]:
weights_array_2008_tech=np.array([2.18892709e-01, 5.75050292e-01, 3.97318093e-02, 1.66125078e-01,
       2.00111131e-04])

## Weight Optimization for TRAVEL Industry

### Dataframe with COVID Closing Prices -Travel

In [75]:
# create new covid and 2008 dataframe to store ckosing prices of each stock
covid_closingprices_ta = pd.DataFrame()

# get closing prices for all tickers
for ticker in tickers_travel_airline:
    covid_closingprices_ta[ticker]=tra_air_covid_ticker['Adj Close'][ticker]
    
# drop time component on index
covid_closingprices_ta.index = covid_closingprices_ta.index.date
    
covid_closingprices_ta.head()

,EXPE,BKNG,AAL,AAP,LUV
2017-11-01,118.678818,1913.060059,46.526157,79.808350,52.382694
2017-11-02,120.098473,1911.140015,46.059338,80.737404,52.382694
2017-11-03,119.651192,1894.489990,46.146866,80.846115,52.528225
2017-11-06,119.583122,1903.000000,46.205212,79.768799,52.896912
2017-11-07,116.306206,1645.719971,45.184052,78.454323,52.266262


### Dataframe with 2008 Closing Prices -Travel

In [78]:
closingprices2008_ta = pd.DataFrame()

# get closing prices for all tickers
for ticker in tickers_travel_airline:
    closingprices2008_ta[ticker]=tra_air_ticker2008['Adj Close'][ticker]
    
# drop time component on index
closingprices2008_ta.index = closingprices2008_ta.index.date
    
closingprices2008_ta.tail()

,EXPE,BKNG,AAL,AAP,LUV
2007-06-25,38.490635,64.500000,27.565760,37.980713,13.836042
2007-06-26,38.516979,64.160004,26.641878,37.456627,13.503979
2007-06-27,38.806786,64.860001,27.971140,39.086025,13.679236
2007-06-28,38.846291,64.269997,29.224987,38.895458,13.771476
2007-06-29,38.582848,68.739998,28.536783,38.619114,13.753030


### Travel Optimized Weights - COVID

In [79]:
# daily returns
ta_covid_dr = covid_closingprices_ta.pct_change().dropna()
ta_returns2008 = closingprices2008_ta.pct_change().dropna()

In [80]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ta_covid = np.zeros((number_ports, len(covid_closingprices_ta.columns)))
returns_array_ta_covid = np.zeros(number_ports)
volatility_array_ta_covid = np.zeros(number_ports)
sharpe_array_ta_covid = np.zeros(number_ports)



for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ta_covid[index,:] = weights

    #expected returns
    returns_array_ta_covid[index] = np.sum((ta_covid_dr.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ta_covid[index] = np.sqrt(np.dot(weights.T, np.dot(ta_covid_dr.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ta_covid[index] = returns_array_ta_covid[index]/volatility_array_ta_covid[index]

In [82]:
sharpe_array_ta_covid.max()

1.250749691176348

In [83]:
index_max_shape_covid_ta = sharpe_array_ta_covid.argmax()
weights_array_covid_ta = all_weights_ta_covid[index_max_shape_covid_ta,:]
weights_array_covid_ta

array([0.07921269, 0.10322484, 0.01349538, 0.76923025, 0.03483684])

In [84]:
weights_array_covid_ta=np.array([0.07921269, 0.10322484, 0.01349538, 0.76923025, 0.03483684])

### Travel Optimized Weights - 2008

In [85]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_ta_2008 = np.zeros((number_ports, len(closingprices2008_ta.columns)))
returns_array_ta_2008 = np.zeros(number_ports)
volatility_array_ta_2008 = np.zeros(number_ports)
sharpe_array_ta_2008 = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_ta_2008[index,:] = weights

    #expected returns
    returns_array_ta_2008[index] = np.sum((ta_returns2008.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_ta_2008[index] = np.sqrt(np.dot(weights.T, np.dot(ta_returns2008.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_ta_2008[index] = returns_array_ta_2008[index]/volatility_array_ta_2008[index]

In [86]:
sharpe_array_ta_2008.max()

2.083246114873164

In [87]:
index_max_shape_2008_ta = sharpe_array_ta_2008.argmax()
weights_array_2008_ta = all_weights_ta_2008[index_max_shape_2008_ta,:]
weights_array_2008_ta

array([0.14734116, 0.65385975, 0.18062361, 0.01214482, 0.00603065])

In [90]:
weights_array_2008_ta=np.array([0.14734116, 0.65385975, 0.18062361, 0.01214482, 0.00603065])

## Weight Optimization for ENERGY Industry

### Dataframe with Covid Closing Prices - ENERGY

In [91]:
# create new covid and 2008 dataframe to store ckosing prices of each stock
covid_closingprices_nrg = pd.DataFrame()

# get closing prices for all tickers
for ticker in energy_tickers:
    covid_closingprices_nrg[ticker]=energy_covid_ticker['Adj Close'][ticker]
    
# drop time component on index
covid_closingprices_nrg.index = covid_closingprices_nrg.index.date
    
covid_closingprices_nrg.head()

,XOM,BP,COP,CVX,TOT
2017-11-01,70.515472,33.086613,48.129959,100.545921,46.929188
2017-11-02,70.229614,33.013561,48.480999,100.051422,47.221355
2017-11-03,69.935326,32.924267,48.924419,99.756477,47.021023
2017-11-06,70.414566,33.614246,49.580315,101.534889,47.546906
2017-11-07,70.271652,33.671074,49.404804,101.708405,47.446735


### Dataframe with 2008 Closing Prices - ENERGY

In [92]:
closingprices2008_nrg = pd.DataFrame()

# get closing prices for all tickers
for ticker in energy_tickers:
    closingprices2008_nrg[ticker]=energy_ticker2008['Adj Close'][ticker]
    
# drop time component on index
closingprices2008_nrg.index = closingprices2008_nrg.index.date
    
closingprices2008_nrg.tail()

,XOM,BP,COP,CVX,TOT
2007-06-25,52.500305,33.523296,37.768803,49.850029,37.032543
2007-06-26,52.149746,33.661957,36.684723,49.699764,36.748394
2007-06-27,53.207775,33.924938,37.391308,50.414940,36.838379
2007-06-28,53.284252,34.331371,37.139641,50.589203,37.752354
2007-06-29,53.462708,34.493946,37.991432,50.625267,38.349037


### Energy Optimized Weights - COVID

In [93]:
# daily returns
nrg_covid_dr = covid_closingprices_nrg.pct_change().dropna()
nrg_returns2008 = closingprices2008_nrg.pct_change().dropna()

In [94]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_nrg_covid = np.zeros((number_ports, len(covid_closingprices_nrg.columns)))
returns_array_nrg_covid = np.zeros(number_ports)
volatility_array_nrg_covid = np.zeros(number_ports)
sharpe_array_nrg_covid = np.zeros(number_ports)



for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_nrg_covid[index,:] = weights

    #expected returns
    returns_array_nrg_covid[index] = np.sum((nrg_covid_dr.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_nrg_covid[index] = np.sqrt(np.dot(weights.T, np.dot(nrg_covid_dr.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_nrg_covid[index] = returns_array_nrg_covid[index]/volatility_array_nrg_covid[index]

In [95]:
sharpe_array_nrg_covid.max()

0.3752023504481871

In [96]:
index_max_shape_covid_nrg = sharpe_array_nrg_covid.argmax()
weights_array_covid_nrg = all_weights_nrg_covid[index_max_shape_covid_nrg,:]
weights_array_covid_nrg

array([0.00465928, 0.09414364, 0.75722529, 0.12846314, 0.01550865])

In [100]:
weights_array_covid_nrg=np.array([0.00465928, 0.09414364, 0.75722529, 0.12846314, 0.01550865])

### Energy Optimized Weights - 2008

In [98]:
# running 10k simulations on portfolio
number_ports = 100000
all_weights_nrg_2008 = np.zeros((number_ports, len(closingprices2008_nrg.columns)))
returns_array_nrg_2008 = np.zeros(number_ports)
volatility_array_nrg_2008 = np.zeros(number_ports)
sharpe_array_nrg_2008 = np.zeros(number_ports)

np.random.seed(4)

for index in range(number_ports):
    #weights
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    #save weights
    all_weights_nrg_2008[index,:] = weights

    #expected returns
    returns_array_nrg_2008[index] = np.sum((nrg_returns2008.mean()*weights)*252)

    #expected volatiltiy
    volatility_array_nrg_2008[index] = np.sqrt(np.dot(weights.T, np.dot(nrg_returns2008.cov()*252, weights)))

    #sharpe ratio
    sharpe_array_nrg_2008[index] = returns_array_nrg_2008[index]/volatility_array_nrg_2008[index]

In [101]:
sharpe_array_nrg_2008.max()

1.2619428350562292

In [102]:
index_max_shape_2008_nrg = sharpe_array_nrg_2008.argmax()
weights_array_2008_nrg = all_weights_nrg_2008[index_max_shape_2008_nrg,:]
weights_array_2008_nrg

array([0.17478148, 0.00524994, 0.00542368, 0.3866935 , 0.42785141])

In [103]:
weights_array_2008_nrg=np.array([0.17478148, 0.00524994, 0.00542368, 0.3866935 , 0.42785141])